In [1]:
import pandas as pd

from tqdm import tqdm
import json

# 1. Part

In [2]:
df_input = pd.read_excel('./file/2.part.xlsx',
                         header=0,
                         dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)
df_input['Price'] = df_input['Price'].astype(float)

# = = = = = = = = = = = = = = =

list_series, list_column, list_column_description = [], df_input.columns.tolist(), []
for _, series in tqdm(df_input.iterrows(), total=len(df_input), desc='Progress', ncols=77):
    dict_dict = json.loads(series['Json_Description'])
    dict_ = {k: v for dict_ in dict_dict.values() for k, v in dict_.items()}

    for key, value in dict_.items():
        if key in list_column:
            key = f'{key} (Description)'

        if key not in list_column_description:
            list_column_description.append(key)

        series[key] = value

    list_series.append(series)

# = = = = = = = = = = = = = = =

df_output = pd.DataFrame(list_series).sort_values(by=['No'],
                                                  ascending=[True],
                                                  ignore_index=True)
df_output[list_column[:12] + sorted(list_column_description) + list_column[12:]].to_excel('./file/3.part.xlsx', index=False)

print()
print('Done ~')

Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 210.13it/s]


Done ~


# 2. Vehicle

In [3]:
df_input = pd.read_excel('./file/2.compatibility.xlsx',
                         header=0,
                         dtype=str).fillna('')

df_input['Year'] = df_input['Year'].astype(int)

# = = = = = = = = = = = = = = =

list_dict, grouped = [], df_input.groupby('SKU')
for sku, group in tqdm(grouped, desc='Progress', ncols=77):
    df_temp = group.reset_index(drop=True)

    dict_vehicle = {}
    for i in range(len(df_temp)):
        make, model, year = df_temp.loc[i, 'Make'], df_temp.loc[i, 'Model'], df_temp.loc[i, 'Year']

        if make not in dict_vehicle:
            dict_vehicle[make] = {model: [year]}
        elif model not in dict_vehicle[make]:
            dict_vehicle[make][model] = [year]
        elif year not in dict_vehicle[make][model]:
            dict_vehicle[make][model].append(year)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make, dict_model in dict_vehicle.items():
        for model, list_year in dict_model.items():
            year_begin, year_end = min(list_year), max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    # = = = = = = = = = = = = = = =

    list_dict.append({'SKU': sku,
                      'Vehicle': '\n'.join(sorted(list_vehicle))})

# = = = = = = = = = = = = = = =

df_output = pd.DataFrame(list_dict).sort_values(by=['SKU'],
                                                ascending=[True],
                                                ignore_index=True)
df_output.to_excel('./file/3.vehicle.xlsx', index=False)

print()
print('Done ~')

Progress: 100%|████████████████████████████| 10/10 [00:00<00:00, 1259.93it/s]


Done ~
